In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plaid.compression.hourglass_vq import HourglassVQLightningModule
from pathlib import Path
import torch

In [4]:
#@title Load VQVAE model

ckpt_path = "/homefs/home/lux70/storage/plaid/checkpoints/hourglass_vq/2024-03-01T22-21-43/last-v3.ckpt"
vqvae = HourglassVQLightningModule.load_from_checkpoint(ckpt_path)
from plaid.utils import n_params
print(f"{n_params(vqvae):,}")

111,326,208


In [5]:
# load a toy dataloader
from plaid.datasets import CATHShardedDataModule
shard_dir = "/homefs/home/lux70/storage/data/cath/shards"
dm = CATHShardedDataModule(shard_dir=shard_dir)
dm.setup()
train_loader = dm.train_dataloader()

batch = next(iter(train_loader))

In [18]:
# Try forward pass

from plaid.utils import LatentScaler
latent_scaler = LatentScaler()

device = torch.device("cuda")
x = latent_scaler.scale(batch[0])

from plaid.esmfold.misc import batch_encode_sequences
_, mask, _, _, _ = batch_encode_sequences(batch[1])

x = x.to(device)
vqvae = vqvae.to(device)
mask = mask.to(device).bool()

x_recons, loss, log_dict, quant_out = vqvae(x, mask, return_vq_output=True, log_wandb=False) 
print(log_dict)

{'vq_loss': 0.010185823775827885, 'vq_perplexity': tensor(229.1983, device='cuda:0'), 'recons_loss': 0.009069736115634441, 'loss': 0.019255559891462326}


In [19]:
print(quant_out.keys())
print(quant_out['min_encoding_indices'].shape)
print(quant_out['min_encodings'].shape)

dict_keys(['loss', 'z_q', 'perplexity', 'min_encodings', 'min_encoding_indices'])
torch.Size([16384, 1])
torch.Size([16384, 1024])


In [28]:
codebook = quant_out['min_encoding_indices']
codebook = codebook.reshape(-1, x.shape[1])
print(codebook.shape)
print(codebook)

torch.Size([256, 64])
tensor([[ 296,  124,  128,  ...,  107,  895,  909],
        [ 444,  336,  123,  ...,  989,  495,  819],
        [ 617,  128,  336,  ...,  368, 1009,  437],
        ...,
        [ 165,  147,  152,  ...,  663,   35,    3],
        [ 680,  504,  698,  ...,  945,   35,  312],
        [ 860,  284,  361,  ...,  319,  604,  296]], device='cuda:0')


In [30]:
from plaid.proteins import LatentToStructure
import py3Dmol

for i in range(10):
    try:
        cath_pdb = 

view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(pdbstr)
view.setStyle({"model": -1}, {"cartoon": {"color": "green"}})
view.zoomTo()
view.show()


latent_to_structure = LatentToStructure()
latent_to_structure.to(device)
x_recons_unnormed = latent_scaler.unscale(x_recons)
pdbs, metrics = latent_to_structure.to_structure(x_recons_unnormed, batch[1])

In [42]:
def id_to_path(pid):
    return f"/homefs/home/lux70/storage/data/cath/dompdb/{pid}"

import os
os.path.isfile(id_to_path(batch[-1][0]))

True

In [46]:
from plaid.utils import view_py3Dmol
import py3Dmol


for i in range(10):
    # load orig structure
    try:
        with open(id_to_path(batch[-1][i]), "r") as f:
            orig_struct_str = f.read()
    except Exception as e:
        print(e)
        continue

    # view overlay  
    view = py3Dmol.view(width=400, height=300)
    view.addModelsAsFrames(pdbs[0])
    view.addModelsAsFrames(orig_struct_str)
    view.setStyle({"model": 0}, {"cartoon": {"color": "orange"}})
    view.setStyle({"model": 1}, {"cartoon": {"color": "blue"}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [35]:
batch[-1][0]

'4kvmA01'

True